# Train Like Bonito
https://github.com/nanoporetech/bonito/blob/master/notebooks/bonito-train.ipynb

In [2]:
import os
import sys
import time
import random
from datetime import datetime
from itertools import starmap
from pathlib import Path

import numpy as np
import pandas as pd
import toml
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.files.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from bonito.model import Model
from bonito.util import accuracy
from bonito.training import ChunkDataSet
from bonito.decode import decode, decode_ref

import jkbc.files.torch_files as f

ModuleNotFoundError: No module named 'bonito'

In [4]:
# Sections of squiggle that correspond with the target reference sequence
data = f.load_training_data('/basecaller-p10/data/feather-files/Range0-10000-FixLabelLen1024-winsize4096')

full_chunks = load_np("chunks.npy")
full_chunk_lengths = load_np("chunk_lengths.npy")
full_targets = load_np("references.npy")
full_target_lengths = load_np("reference_lengths.npy")

print(full_chunks.shape)
full_chunks = data.x
full_chunk_lengths = data.x_lengths
full_targets = data.y
full_target_lengths = data.y_lengths

# The structure of the model is defined using a config file.
# This will make sense to those familar with QuartzNet
# https://arxiv.org/pdf/1910.10261.pdf).
quartznet_config = toml.load("../models/bonito/quartznet5x5.toml")

#### Training options
Default options are set, and ranges are sensible, but most combinations of settings are untested.

The default settings will train on a small amount of data (1000 signal chunks) for a small number of epochs (20). This is unlikely to produce an accurate generalisable model, but will train relatively quickly.

After modifying this cell, Runtime -> Run after, so that all cells between this one and the main train looping will be run in accordance with new setting.

A train_proportion of 0.90 will use 90% of the data for training and 10% for validation.

No dropout is applied by default, but in order to avoid overfitting on small data sets it may be necessary to apply dropout (e.g. of 0.5), or other regularisation techniques.

In [9]:
model_savepath = Path("train_like_bonito/models")
learning_rate = 0.001 #@param {type:"number"}
random_seed = 25 #@param {type:"integer"}
epochs = 20 #@param {type:"slider", min:1, max:1000, step:1}
batch_size = 16 #@param [2, 4, 8, 16, 28] {type:"raw"}
num_chunks = 10000 #@param [10, 100, 1000, 10000, 100000] {type:"raw"}
train_proportion = 0.80 #@param type:"slider", min:0.8, max:1000, step:1
dropout = 0.0 #@param {type:"slider", min:0.0, max:0.8}

In [10]:
# Initialise random libs and setup cudnn
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# we exploit GPU for training
device = torch.device("cuda")

#### Prepare data according to values set in the 'Training options

In [11]:
# subset
chunks = full_chunks[:num_chunks]
chunk_lengths = full_chunk_lengths[:num_chunks]
targets = full_targets[:num_chunks]
target_lengths = full_target_lengths[:num_chunks]

# shuffle
shuf = np.random.permutation(chunks.shape[0])
chunks = chunks[shuf]
chunk_lengths = chunk_lengths[shuf]
targets = targets[shuf]
target_lengths = target_lengths[shuf]

split = np.floor(chunks.shape[0] * train_proportion).astype(np.int32)

In [12]:
for b in quartznet_config['block']:
    b['dropout'] = dropout
quartznet_config

{'model': 'QuartzNet',
 'labels': {'labels': ['N', 'A', 'C', 'G', 'T']},
 'input': {'features': 1},
 'encoder': {'activation': 'relu'},
 'block': [{'filters': 256,
   'repeat': 1,
   'kernel': [33],
   'stride': [3],
   'dilation': [1],
   'dropout': 0.0,
   'residual': False,
   'separable': False},
  {'filters': 256,
   'repeat': 5,
   'kernel': [33],
   'stride': [1],
   'dilation': [1],
   'dropout': 0.0,
   'residual': True,
   'separable': True},
  {'filters': 256,
   'repeat': 5,
   'kernel': [39],
   'stride': [1],
   'dilation': [1],
   'dropout': 0.0,
   'residual': True,
   'separable': True},
  {'filters': 512,
   'repeat': 5,
   'kernel': [51],
   'stride': [1],
   'dilation': [1],
   'dropout': 0.0,
   'residual': True,
   'separable': True},
  {'filters': 512,
   'repeat': 5,
   'kernel': [63],
   'stride': [1],
   'dilation': [1],
   'dropout': 0.0,
   'residual': True,
   'separable': True},
  {'filters': 512,
   'repeat': 5,
   'kernel': [75],
   'stride': [1],
   'di

## Training and test functions

In [13]:

# 'Connectionist Temporal Classification' (CTC) loss fuction
# https://distill.pub/2017/ctc/
criterion = nn.CTCLoss(reduction='mean')

def train(log_interval, model, device, train_loader,
          optimizer, epoch, use_amp=False):

    t0 = time.perf_counter()
    chunks = 0

    model.train()

    sys.stderr.write("\n" + "Training epoch: " + str(epoch) + "\n")
    progress_bar = tqdm(total=len(train_loader), leave=True, ncols=100)

    for batch_idx, (data, out_lengths, target, lengths) in enumerate(train_loader, start=1):

        optimizer.zero_grad()

        chunks += data.shape[0]

        data = data.to(device)
        target = target.to(device)

        # forward pass
        log_probs = model(data)    

        # calculate loss
        loss = criterion(log_probs.transpose(0, 1), target, out_lengths / model.stride, lengths)

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
        progress_bar.refresh()
        progress_bar.update(1)
        progress_bar.set_description("Loss: " + str(loss.item()))
        sys.stderr.flush()        

    progress_bar.close()

    return loss.item(), time.perf_counter() - t0


def test(model, device, test_loader):

    model.eval()
    test_loss = 0
    predictions = []
    prediction_lengths = []

    with torch.no_grad():
        for batch_idx, (data, out_lengths, target, lengths) in enumerate(test_loader, start=1):
            data, target = data.to(device), target.to(device)
 
            # forward pass
            log_probs = model(data)
 
            # calculate loss
            test_loss += criterion(log_probs.transpose(1, 0), target, out_lengths / model.stride, lengths)

            # accumulate output probabilities
            predictions.append(torch.exp(log_probs).cpu())
            prediction_lengths.append(out_lengths / model.stride)

    predictions = np.concatenate(predictions)
    lengths = np.concatenate(prediction_lengths)

    # convert probabilities to sequences
    references = [decode_ref(target, model.alphabet) for target in test_loader.dataset.targets]
    sequences = [decode(post[:n], model.alphabet) for post, n in zip(predictions, lengths)]

    # align predicted sequences with true sequences and calculate accuracy
    if all(map(len, sequences)):
        accuracies = list(starmap(accuracy, zip(references, sequences)))
    else:
        accuracies = [0]

    # measure average accuracies over entire set of validation chunks
    mean = np.mean(accuracies)
    median = np.median(accuracies)

    return test_loss.item() / batch_idx, mean, median

## Main Training Loop

In [19]:
#@title Set experiment name
experiment_name = 'bonito_on_our_data_007' #@param {type:"string"}

In [20]:
def kaiming_uniformise_(model):
    for l in model.modules():
        if isinstance(l, nn.Conv1d):
            nn.init.kaiming_uniform_(l.weight)

In [ ]:
# prevent overwriting of data
workdir = os.path.join(model_savepath, experiment_name)
if os.path.isdir(workdir):
    raise IOError('{} already exists. Select an alternative model_savepath.'.format(workdir))
os.makedirs(workdir)

# data generators
train_dataset = ChunkDataSet(chunks[:split], chunk_lengths[:split],
                             targets[:split], target_lengths[:split])
test_dataset = ChunkDataSet(chunks[split:], chunk_lengths[split:],
                            targets[split:], target_lengths[split:])

# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         num_workers=4, pin_memory=True)

# load bonito model
model = Model(quartznet_config)
model.to(device)
model.train()

# set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), amsgrad=True, lr=learning_rate)
schedular = CosineAnnealingLR(optimizer, epochs * len(train_loader))

# report loss every 
interval = 500 / num_chunks
log_interval = np.floor(len(train_dataset) / batch_size * interval)

exp_config = os.path.join(workdir, "experimental.log")
with open(exp_config, 'a') as c:
    c.write('Num training chunks: {}'.format(num_chunks) + '\n')
    c.write('learning rate: {}'.format(learning_rate) + '\n')
    c.write('random seed: {}'.format(random_seed) + '\n')
    c.write('epochs: {}'.format(epochs) + '\n')
    c.write('batch_size: {}'.format(batch_size) + '\n')
    c.write('train proportion: {}'.format(train_proportion) + '\n')
    c.write('dropout: {}'.format(dropout) + '\n')

# DataFrame to store training logging information
training_results = pd.DataFrame()

for epoch in range(1, epochs + 1):

    train_loss, duration = train(log_interval, model, device,
                                 train_loader, optimizer, epoch)
    
    test_loss, mean, median = test(model, device, test_loader)

    # collate training and validation metrics
    epoch_result = pd.DataFrame(
        {'time':[datetime.today()],
         'duration':[int(duration)],
         'epoch':[epoch],
         'train_loss':[train_loss],
         'validation_loss':[test_loss], 
         'validation_mean':[mean],
         'validation_median':[median]})
    
    # save model weights
    weights_path = os.path.join(workdir, "weights_%s.tar" % epoch)
    torch.save(model.state_dict(), weights_path)

    # update log file
    log_path = os.path.join(workdir, "training.log")
    epoch_result.to_csv(log_path, mode='a', sep='\t', index=False)

    display(epoch_result)
    training_results = training_results.append(epoch_result)

    schedular.step()

display(training_results)


Training epoch: 1
Loss: 0.7485458850860596: 100%|███████████████████████████████████| 500/500 [01:50<00:00,  4.54it/s]


,time,duration,epoch,train_loss,validation_loss,validation_mean,validation_median
0,2020-04-07 12:31:53.621902,110,1,0.748546,0.751881,73.506264,73.592311



Training epoch: 2
Loss: 0.4637991189956665: 100%|███████████████████████████████████| 500/500 [01:50<00:00,  4.55it/s]


,time,duration,epoch,train_loss,validation_loss,validation_mean,validation_median
0,2020-04-07 12:33:52.519944,110,2,0.463799,0.50625,83.730111,83.879781



Training epoch: 3
Loss: 0.38080522418022156: 100%|██████████████████████████████████| 500/500 [01:50<00:00,  4.54it/s]


...


Training epoch: 6
Loss: 0.17058908939361572: 100%|██████████████████████████████████| 500/500 [01:50<00:00,  4.53it/s]


,time,duration,epoch,train_loss,validation_loss,validation_mean,validation_median
0,2020-04-07 12:41:47.953550,110,6,0.170589,0.260316,92.265061,92.878338



Training epoch: 7
Loss: 0.19782748818397522: 100%|██████████████████████████████████| 500/500 [01:50<00:00,  4.54it/s]


,time,duration,epoch,train_loss,validation_loss,validation_mean,validation_median
0,2020-04-07 12:43:47.059304,110,7,0.197827,0.254359,92.58659,93.133047



Training epoch: 8
Loss: 0.15273253619670868:  51%|█████████████████▍                | 256/500 [00:56<00:53,  4.52it/s]

## Tries (after 1 epoch): 

With Kaiming:

1: 0.9324800968170166

2: 0.9282790422439575

Without Kaiming:

1: 0.7507233619689941

Without Kaiming, dropout=0.5:

1: 1.2488034963607788

I am unsure why the runs differ with same settings when we use seed.